> a woman in Staten Island came to pick it up... She said “I only have ten or eleven cats, and anyway they are getting quite old now...

I visually inspected the "products" table and found the pet food section.
They specifically sell senior cat food!

The goal is:
- find people in Staten Island (FIXME: explain how this gets removed in the speedrun)
- find which one buys the most senior cat food

In [1]:
import polars as pl

customers = pl.read_csv("harder-data/noahs-customers.csv",try_parse_dates=True)
orders_items = pl.read_csv("harder-data/noahs-orders_items.csv",try_parse_dates=True)
orders = pl.read_csv("harder-data/noahs-orders.csv",try_parse_dates=True)
products = pl.read_csv("harder-data/noahs-products.csv",try_parse_dates=True)

In [2]:
# Find the senior cat food items
senior_cat_food_products = products.filter(
    pl.col("desc").str.contains("Senior Cat Food")
)

# Find who has bought the most senior cat food among all customers
orders_items_with_cat_food = orders_items.join(
    senior_cat_food_products, on="sku", how="inner"
)
orders_with_cat_food = orders.join(
    orders_items_with_cat_food, on="orderid", how="inner"
)
cat_food_mvp = (
    orders_with_cat_food.group_by("customerid")
    .agg(pl.col("qty").sum())
    .sort("qty", descending=True)
    .limit(1)
    .select("customerid")
    .item()
)

print(customers.filter(pl.col("customerid") == cat_food_mvp).select("name", "phone"))


shape: (1, 2)
┌────────────────┬──────────────┐
│ name           ┆ phone        │
│ ---            ┆ ---          │
│ str            ┆ str          │
╞════════════════╪══════════════╡
│ Theresa Carter ┆ 347-835-2358 │
└────────────────┴──────────────┘
